In [1]:
import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [2]:
#path variables
game_url = "chrome://dino"
chrome_driver_path = "c:/abc/chromedriver.exe"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

#scripts
#create id for canvas for faster selection from DOM
init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

#get image from canvas
getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
return canvasRunner.toDataURL().substring(22)"





In [3]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* restart() : sends a signal to browser-javascript to restart the game
* press_up(): sends a single to press up get to the browser
* get_score(): gets current game score from javascript variables.
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.get('chrome://dino')
        self._driver.execute_script("Runner.config.ACCELERATION=0")
        self._driver.execute_script(init_script)
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) # the javascript object is of type array with score in the formate[1,0,0] which is 100.
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

In [4]:
class DinoAgent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game; 
        self.jump(); #to start the game, we need to jump once
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [5]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine 
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = self._game.get_score() 
        reward = 0.1
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
        image = grab_screen(self._game._driver) 
        self._display.send(image) #display the image on screen
        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score # log the score when game is over
            self._game.restart()
            reward = -1
            is_over = True
        return image, reward, is_over #return the Experience tuple

In [6]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    if not os.path.exists("./objects"):
        os.mkdir("./objects")
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #RGB to Grey Scale
    image = image[:300, :500] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [7]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [8]:
#game parameters
ACTIONS = 2 # possible actions: jump, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 #We stack 4 frames

In [9]:
# training variables saved as checkpoints to filesystem to resume training from the same step# train 
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [10]:
'''Call only once to init file structure
'''
init_cache()

In [11]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [12]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [13]:
#main function
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_sate(dino,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [14]:
playGame (observe=False);

Now we build the model
We finish building the model
fps: 1.4103563901259344
TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 12.819757010774051
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.737889286806357
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 12.986305611200729
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 12.986225195909357
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 15.872184064634538
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
fps: 16.392452358247223
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.27533173720732
TIMESTEP 8 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 20.832256366192006
TIMESTEP 9 

fps: 21.73777662606893
TIMESTEP 76 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.737889286806357
TIMESTEP 77 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.737889286806357
TIMESTEP 78 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.27543965547851
TIMESTEP 79 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 20.83204942907236
TIMESTEP 80 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
fps: 21.737889286806357
TIMESTEP 81 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 16.128154548355962
TIMESTEP 82 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
fps: 21.27533173720732
TIMESTEP 83 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
fps: 20.832152897118277
TIMESTEP 84 / STATE obse

TIMESTEP 138 / STATE explore / EPSILON 0.09996303699999995 / ACTION 0 / REWARD 0.1 / Q_MAX  2.71934 / Loss  0.122187934816
fps: 6.369066256873127
TIMESTEP 139 / STATE explore / EPSILON 0.09996203799999995 / ACTION 0 / REWARD 0.1 / Q_MAX  2.37618 / Loss  0.143545269966
fps: 6.369066256873127
TIMESTEP 140 / STATE explore / EPSILON 0.09996103899999995 / ACTION 0 / REWARD 0.1 / Q_MAX  3.78933 / Loss  0.0426059663296
fps: 5.916820665897376
TIMESTEP 141 / STATE explore / EPSILON 0.09996003999999994 / ACTION 1 / REWARD 0.1 / Q_MAX  2.61015 / Loss  0.111210919917
fps: 5.813617349215898
TIMESTEP 142 / STATE explore / EPSILON 0.09995904099999994 / ACTION 0 / REWARD 0.1 / Q_MAX  2.40603 / Loss  0.0343694128096
fps: 6.328751331597101
TIMESTEP 143 / STATE explore / EPSILON 0.09995804199999994 / ACTION 0 / REWARD 0.1 / Q_MAX  2.95319 / Loss  0.0623238161206
fps: 6.493132701715276
TIMESTEP 144 / STATE explore / EPSILON 0.09995704299999994 / ACTION 0 / REWARD 0.1 / Q_MAX  3.06366 / Loss  0.21756625175

fps: 3.520926757607555
TIMESTEP 193 / STATE explore / EPSILON 0.09990809199999987 / ACTION 0 / REWARD 0.1 / Q_MAX  3.7065 / Loss  0.085218295455
fps: 5.952043684890937
TIMESTEP 194 / STATE explore / EPSILON 0.09990709299999986 / ACTION 0 / REWARD 0.1 / Q_MAX  3.45737 / Loss  0.930145084858
fps: 6.369056585436964
TIMESTEP 195 / STATE explore / EPSILON 0.09990609399999986 / ACTION 0 / REWARD 0.1 / Q_MAX  2.49183 / Loss  0.149331599474
fps: 6.451246935341473
TIMESTEP 196 / STATE explore / EPSILON 0.09990509499999986 / ACTION 0 / REWARD 0.1 / Q_MAX  3.44105 / Loss  0.841553330421
fps: 6.409888301197068
TIMESTEP 197 / STATE explore / EPSILON 0.09990409599999986 / ACTION 0 / REWARD 0.1 / Q_MAX  3.27801 / Loss  0.103698290884
fps: 5.78001697774152
TIMESTEP 198 / STATE explore / EPSILON 0.09990309699999986 / ACTION 0 / REWARD 0.1 / Q_MAX  2.96313 / Loss  0.0543018132448
fps: 5.88202032894294
TIMESTEP 199 / STATE explore / EPSILON 0.09990209799999986 / ACTION 0 / REWARD 0.1 / Q_MAX  2.39609 / L

TIMESTEP 247 / STATE explore / EPSILON 0.09985414599999978 / ACTION 0 / REWARD 0.1 / Q_MAX  1.59768 / Loss  0.0706109404564
fps: 5.34729257157573
TIMESTEP 248 / STATE explore / EPSILON 0.09985314699999978 / ACTION 1 / REWARD 0.1 / Q_MAX  3.03396 / Loss  0.0991578325629
----------Random Action----------
fps: 3.9838340626368334
TIMESTEP 249 / STATE explore / EPSILON 0.09985214799999978 / ACTION 0 / REWARD 0.1 / Q_MAX  1.88831 / Loss  0.631474077702
fps: 3.086307325070419
TIMESTEP 250 / STATE explore / EPSILON 0.09985114899999978 / ACTION 1 / REWARD -1 / Q_MAX  3.08559 / Loss  0.0506854914129
fps: 3.1744320078227353
TIMESTEP 251 / STATE explore / EPSILON 0.09985014999999978 / ACTION 0 / REWARD 0.1 / Q_MAX  2.77287 / Loss  0.0428456813097
fps: 4.291603228394152
TIMESTEP 252 / STATE explore / EPSILON 0.09984915099999978 / ACTION 0 / REWARD 0.1 / Q_MAX  3.07987 / Loss  0.0397121421993
fps: 6.409888301197068
TIMESTEP 253 / STATE explore / EPSILON 0.09984815199999977 / ACTION 0 / REWARD 0.1 / 

fps: 6.409888301197068
TIMESTEP 303 / STATE explore / EPSILON 0.0997982019999997 / ACTION 0 / REWARD 0.1 / Q_MAX  1.86754 / Loss  0.330304682255
fps: 6.409888301197068
TIMESTEP 304 / STATE explore / EPSILON 0.0997972029999997 / ACTION 0 / REWARD 0.1 / Q_MAX  2.05316 / Loss  0.110023260117
fps: 6.409888301197068
TIMESTEP 305 / STATE explore / EPSILON 0.0997962039999997 / ACTION 0 / REWARD 0.1 / Q_MAX  1.86959 / Loss  0.106744147837
fps: 6.451246935341473
TIMESTEP 306 / STATE explore / EPSILON 0.09979520499999969 / ACTION 0 / REWARD 0.1 / Q_MAX  1.20923 / Loss  0.140210419893
fps: 6.451246935341473
TIMESTEP 307 / STATE explore / EPSILON 0.09979420599999969 / ACTION 0 / REWARD 0.1 / Q_MAX  3.01818 / Loss  0.338230550289
----------Random Action----------
fps: 7.142438231923397
TIMESTEP 308 / STATE explore / EPSILON 0.09979320699999969 / ACTION 0 / REWARD 0.1 / Q_MAX  3.00581 / Loss  0.0178816914558
fps: 6.369066256873127
TIMESTEP 309 / STATE explore / EPSILON 0.09979220799999969 / ACTION 0

TIMESTEP 359 / STATE explore / EPSILON 0.09974225799999961 / ACTION 0 / REWARD 0.1 / Q_MAX  3.08172 / Loss  0.120293751359
fps: 6.409888301197068
TIMESTEP 360 / STATE explore / EPSILON 0.09974125899999961 / ACTION 0 / REWARD 0.1 / Q_MAX  3.51618 / Loss  0.0484970100224
fps: 5.813625407333175
TIMESTEP 361 / STATE explore / EPSILON 0.09974025999999961 / ACTION 1 / REWARD 0.1 / Q_MAX  2.95249 / Loss  0.137399092317
fps: 6.369046914030172
TIMESTEP 362 / STATE explore / EPSILON 0.09973926099999961 / ACTION 0 / REWARD 0.1 / Q_MAX  3.29603 / Loss  0.0824349820614
fps: 6.451256858001116
TIMESTEP 363 / STATE explore / EPSILON 0.0997382619999996 / ACTION 0 / REWARD 0.1 / Q_MAX  2.57117 / Loss  0.0274202488363
fps: 5.813617349215898
TIMESTEP 364 / STATE explore / EPSILON 0.0997372629999996 / ACTION 1 / REWARD 0.1 / Q_MAX  3.23346 / Loss  0.0488679111004
fps: 5.847618808616596
TIMESTEP 365 / STATE explore / EPSILON 0.0997362639999996 / ACTION 1 / REWARD 0.1 / Q_MAX  3.10323 / Loss  0.0179761648178

TIMESTEP 415 / STATE explore / EPSILON 0.09968631399999953 / ACTION 0 / REWARD 0.1 / Q_MAX  2.38589 / Loss  0.133085682988
fps: 6.451395778441226
TIMESTEP 416 / STATE explore / EPSILON 0.09968531499999952 / ACTION 0 / REWARD 0.1 / Q_MAX  2.58793 / Loss  0.0145416622981
fps: 7.142450394730568
TIMESTEP 417 / STATE explore / EPSILON 0.09968431599999952 / ACTION 0 / REWARD 0.1 / Q_MAX  3.30909 / Loss  0.0791775509715
fps: 7.142450394730568
TIMESTEP 418 / STATE explore / EPSILON 0.09968331699999952 / ACTION 0 / REWARD 0.1 / Q_MAX  3.0527 / Loss  0.0929294675589
fps: 7.091789236614246
TIMESTEP 419 / STATE explore / EPSILON 0.09968231799999952 / ACTION 0 / REWARD 0.1 / Q_MAX  2.1069 / Loss  0.376366615295
fps: 6.369066256873127
TIMESTEP 420 / STATE explore / EPSILON 0.09968131899999952 / ACTION 0 / REWARD 0.1 / Q_MAX  3.37916 / Loss  0.318995207548
fps: 7.142559861859213
TIMESTEP 421 / STATE explore / EPSILON 0.09968031999999952 / ACTION 0 / REWARD 0.1 / Q_MAX  1.88705 / Loss  0.0734240561724

TypeError: can only join an iterable